In [10]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os


In [13]:
np.float32 is np.single

True

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")
create_folder_if_not_exists("files")

In [ ]:
def get_A1(n, variable_type):
    A_ = np.zeros((n, n), dtype=variable_type)
    N = np.zeros(n, dtype=variable_type)
    for i in range(n):
        tmp = variable_type(i)
        N[i] = tmp
    for i in N:
        for j in N:
            index_i = int(i)
            index_j = int(j)
            if index_i == 1:
                A_[index_i][index_j] = variable_type(1)
            else:
                one = variable_type(1)
                A_[index_i][index_j] = one / ((i + one) + (j + one) - one)
    return A_


get_A1(3, np.float64)


In [ ]:
def get_A2(n, variable_type):
    A_ = np.zeros((n, n), dtype=variable_type)
    N = np.zeros(n, dtype=variable_type)
    one = variable_type(1)
    two = variable_type(2)
    for i in range(n):
        tmp = variable_type(i)
        N[i] = tmp
    for i in N:
        for j in N:
            index_i = int(i)
            index_j = int(j)
            if j >= i:
                A_[index_i][index_j] = (two * (i + one)) / (j + one)
            else:
                A_[index_i][index_j] = (two * (j + one)) / (i + one)
    return A_


get_A2(5, np.float64)

In [ ]:
def get_random_sign():
    return np.random.choice([-1, 1])


def get_x(n, variable_type):
    x_ = np.zeros(n, dtype=variable_type)
    for i in range(n):
        x_[i] = variable_type(variable_type(get_random_sign()))
    return x_


def get_error(correct, actual):
    if len(correct) != len(actual):
        print("Error: vectors have different lengths")
        return
    error = -np.inf
    for i in range(len(correct)):
        tmp = np.abs(correct[i] - actual[i])
        if tmp > error:
            error = tmp
    return error

In [ ]:
import csv


def save_to_csv(filename, data):
    filename = "files/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)


def read_from_csv(filename, variable_type):
    filename = "files/" + filename + ".csv"
    with open(filename, newline='') as file:
        reader = csv.reader(file)
        data = list(reader)
        for arr in range(len(data)):
            data[arr] = np.array(data[arr], dtype=variable_type)
    return data


def generate_data():
    sizes = [i for i in range(3, 20)]
    sizes.append(50)
    sizes.append(100)
    sizes.append(200)
    xes = []
    for size in sizes:
        xes.append(get_x(size, np.float64))
    return save_to_csv("xes", xes)

In [ ]:
def get_b(A, x, n, variable_type):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(n):
            b[i] += A[i][j] * x[j]
        print()
    return b

In [16]:
#write a function which is given a matrix A and a vector b and returns the solution vector x
def gauss(A, b, variable_type):
    n = len(A)
    x = np.zeros(n, dtype=variable_type)
    #print(A)
    for i in range(n):
        for j in range(i + 1, n):
            if A[i][i] == variable_type(0):
                print(f"Division by 0, n = {n}, variable_type = {variable_type}, A[i][i] = {A[i][i]}")
                return
            m = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= m * A[i][k]
            b[j] -= m * b[i]

    #print(b)
    #print(A)
    for i in range(n - 1, -1, -1):
        if A[i][i] == variable_type(0):
            print("Division by 0")
            return
        x[i] = b[i] / A[i][i]
        for j in range(i - 1, -1, -1):
            b[j] -= A[j][i] * x[i]
    #print(b)
    return x

**Wykonanie**

In [21]:
def draw_plot(domain, float32_results, float64_results, title, x_label="n", y_label="wartości błędu", scale_type=None,
              filename=None):
    fig, ax = plt.subplots(figsize=(15, 7.5))
    if scale_type is not None:
        ax.set_yscale(scale_type)
    plt.scatter(x=domain, y=float32_results, label="float32")
    plt.scatter(x=domain, y=float64_results, label="float64")
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig(f"plots/{filename}.png")
    plt.show()


def perform_tests(get_matrix_fn, method):
    def run_test_gauss(correct_x, variable_type):
        nonlocal get_matrix_fn
        n = len(correct_x)
        A = get_matrix_fn(n, variable_type)
        b = get_b(A, correct_x, n, variable_type)
        x_result = gauss(A, b, variable_type)
        error = get_error(correct_x, x_result)
        return error

    xes_32 = read_from_csv("xes", np.float32)
    xes_64 = read_from_csv("xes", np.float64)
    if len(xes_32) != len(xes_64):
        print("Error: vectors have different lengths")
        return
    errors_32 = []
    errors_64 = []
    domain = []
    for x in range(len(xes_32)):
        if len(xes_32[x]) != len(xes_64[x]):
            print("Error: vectors have different lengths")
            return
        domain.append(len(xes_32[x]))
        errors_32.append(run_test_gauss(xes_32[x], np.float32))
        errors_64.append(run_test_gauss(xes_64[x], np.float64))
    draw_plot(domain=domain, float32_results=errors_32, float64_results=errors_64,
              title=f"Wartości błędów, skala liniowa", filename=f"gauss_linear_A1")
    draw_plot(domain=domain, float32_results=errors_32, float64_results=errors_64,
              title=f"Wartości błędów, skala logarytmiczna", scale_type="log", filename=f"gauss_log_A1")


perform_tests(get_A1, "tmp")

asuufbasiufaefe get_A1
